# Part I - Airline delay prediction - Data Wrangling
## by Juanita Smith

## Introduction
Have you ever been stuck in an airport because your flight was delayed or cancelled and wondered if you could have predicted it if you'd had more data? This is our chance to find out.

This analysis will be focused on predicting flight delays or cancellations.

> This dataset reports flights in the United States, including carriers, arrival and departure delays, and reasons for delays, from 1987 to 2008. The data is collected by the Office of Airline Information, Bureau of Transportation Statistics (BTS).
> - See more information from the data expo challenge in 2009 [here](https://community.amstat.org/jointscsg-section/dataexpo/dataexpo2009).
> - See a full description of the features [here](https://www.transtats.bts.gov/DatabaseInfo.asp?QO_VQ=EFD&Yv0x=D.)
> - Data can be downloaded from [here](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/HG7NV7).

Dictionary:
1) Year 1987-2008 
2) Month 1-12 
3) DayofMonth 1-31 
4) DayOfWeek 1 (Monday) - 7 (Sunday) 
5) DepTime actual departure time (local, hhmm) 
6) CRSDepTime scheduled departure time (local, hhmm) 
7) ArrTime actual arrival time (local, hhmm) 
8) CRSArrTime scheduled arrival time (local, hhmm) 
9) UniqueCarrier unique carrier code 
10) FlightNum flight number 
11) TailNum plane tail number 
12) ActualElapsedTime in minutes 
13) CRSElapsedTime in minutes 
14) AirTime in minutes 
15) ArrDelay arrival delay, in minutes 
16) DepDelay departure delay, in minutes 
17) Origin origin IATA airport code 
18) Destination IATA airport code 
19) Distance in miles 
20) TaxiIn - The time elapsed between wheels down and arrival at the destination airport gate in minutes
21) TaxiOut - The time elapsed between departure from the origin airport gate and wheels off in minutes
22) Cancelled was the flight cancelled? 
23) CancellationCode reason for cancellation (A = carrier, B = weather, C = NAS, D = security) 
24) Diverted 1 = yes, 0 = no 
25) CarrierDelay in minutes
26) WeatherDelay in minutes 
27) NASDelay in minutes 
28) SecurityDelay in minutes 
29) LateAircraftDelay in minutes


**Important to note:** According to the documentation, a late flight is defined as a flight arriving or departing 15 minutes or more after the scheduled time.

>**Rubric Tip**: Your code should not generate any errors, and should use functions, loops where possible to reduce repetitive code. Prefer to use functions to reuse code statements.

> **Rubric Tip**: Document your approach and findings in markdown cells. Use comments and docstrings in code cells to document the code functionality.

>**Rubric Tip**: Markup cells should have headers and text that organize your thoughts, findings, and what you plan on investigating next.  





In [1]:
# import all packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import glob

# clear the garbage to free memory as we are working with huge datasets
import gc 

# import warnings
# warnings.filterwarnings("ignore")

# Import custom modules
from src.utils import reduce_mem_usage, create_folder, convert_time

# set plots to be embedded inline
%matplotlib inline

# suppress matplotlib user warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="matplotlib")

# use high resolution if this project is run on an apple device
%config InlineBackend.figure_format='retina'

# Make your Jupyter Notebook wider
from IPython.display import display, HTML
display(HTML('<style>.container { width:80% !important; }</style>'))

# environment settings
# display all columns and rows during visual inspection
pd.options.display.max_columns = None
pd.options.display.max_rows = None


# stop scientific notation on graphs
pd.options.display.float_format = '{:.0f}'.format

In [2]:
sns.set_style("whitegrid")
BASE_COLOR = sns.color_palette()[0]

In [3]:
FILE_NAME_RAW = '../data/flights_raw.pkl'
FILE_NAME_CLEAN = '../data/flights_clean.pkl'

<span style="color:blue; font-size:30px; line-height:40px; font-family: Calibri;">Data Wrangling</span>

<span style="color:blue; font-size:25px; line-height:30px; font-family: Calibri;">Step 1: Data Gathering</span>

Compressed data in format 'bz2' was downloaded in bulk from above link, one file per year from 1987 - 2008. Uncompress each file, and merge the data into one big dataframe.
Due to data size, only read the last 5 years of data

In [4]:
# get the last 5 years of saved archived data files from 1987 - 2008

start_time = time.time()

# Get CSV files list from a folder
path = '../data/dataverse_files'
csv_files = glob.glob(path + "/*.csv.bz2")
csv_files.sort()

# get last 5 years of data
flights = pd.concat((pd.read_csv(f, compression='bz2', encoding='iso-8859-1') for f in csv_files[-5:]))
flights.reset_index(inplace=True, drop=True)

end_time = time.time()
print('Executing time: {} min'.format((end_time - start_time)/60))

Executing time: 2.3076398332913715 min


In [5]:
flights.head(3)

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2004,1,12,1,623,630,901,915,UA,462,N805UA,98,105,80,-14,-7,ORD,CLT,599,7,11,0,NaN,0,0,0,0,0,0
1,2004,1,13,2,621,630,911,915,UA,462,N851UA,110,105,78,-4,-9,ORD,CLT,599,16,16,0,NaN,0,0,0,0,0,0
2,2004,1,14,3,633,630,920,915,UA,462,N436UA,107,105,88,5,3,ORD,CLT,599,4,15,0,NaN,0,0,0,0,0,0


In [6]:
# First, as this is a large dataset, lets reduce the memory of the dataset to make further analysis easier
flights = reduce_mem_usage(flights)


Triggering memory optimization.......

Memory usage of dataframe is 6915.07 MB
Memory usage after optimization is: 2473.93 MB
Decreased by 64.2%


In [7]:
# save the consolidated raw file 
flights.to_pickle(FILE_NAME_RAW)

<span style="color:blue; font-size:25px; line-height:30px; font-family: Calibri;">Step 2: Assessing</span>

In [8]:
# # load the consolidated raw file
flights = pd.read_pickle(FILE_NAME_RAW)

In [9]:
# visual inspection using a sample
flights.sample(20)

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
7122936,2004,12,13,1,1449,1500,1734,1752,DL,542,N903DA,105,112,87,-18,-11,SNA,SLC,588,6,12,0,NaN,0,0,0,0,0,0
18464848,2006,8,25,5,1407,1215,2016,1815,WN,1111,N633SW,249,240,229,121,112,OAK,MDW,1844,5,15,0,NaN,0,10,0,9,0,102
30528604,2008,3,9,7,1615,1540,1739,1725,AA,337,N457AA,144,165,126,14,35,LGA,ORD,733,6,12,0,NaN,0,NaN,NaN,NaN,NaN,NaN
27653523,2007,11,5,1,758,800,845,850,WN,609,N346SW,47,50,38,-5,-2,HOU,SAT,192,2,7,0,NaN,0,0,0,0,0,0
29208740,2008,1,13,7,649,700,755,811,EV,4840,N878AS,66,71,53,-16,-11,FLO,ATL,273,6,7,0,NaN,0,NaN,NaN,NaN,NaN,NaN
26033548,2007,8,14,2,2033,1940,2148,2058,UA,1159,N563UA,134,138,107,49,53,DEN,SNA,846,7,20,0,NaN,0,0,43,0,0,6
19521385,2006,9,4,1,41,40,707,649,AS,106,N618AS,326,309,305,18,1,ANC,LAX,2345,9,12,0,NaN,0,0,0,18,0,0
24809399,2007,6,25,1,1511,1455,1614,1552,DL,973,N633DL,63,57,38,22,16,ATL,SAV,215,5,20,0,NaN,0,0,0,6,0,16
6982939,2004,12,14,2,1937,1940,2120,2128,AA,356,N4XDAA,103,108,83,-8,-3,DFW,OMA,583,2,18,0,NaN,0,0,0,0,0,0
24132337,2007,5,21,1,1758,1800,156,155,US,52,N819AW,298,295,276,1,-2,LAS,JFK,2248,10,12,0,NaN,0,0,0,0,0,0


>**Observations from visual inspection:**

    - All time columns `DepTime, CRSDepTime, ArrTime, CRSArrTime with format hhmm, was converted to a number, dropping leading zero's
    - `Year, month, DayOfMonth` together make up a date field which can be used for time series investigations, to be considered during feature engineering
    - Column `Cancelled` looks like it could be a boolean field
    - ArrDelay and DepDelay sometimes have negative numbers, I assume these are not delays but early departures and arrivals - to be further investigated
    - Looks like the reason for delay features at the end of the dataset, represent a breakdown of the total delay in arrivals (ArrDelay) - to be further investigated
    - Column headings start with a capital letter
    - When `CancellationCode` is 0, means flight was not cancelled, we see NaN as value in CancellationCode. It's not missing as such, but rather an indication of a cancelled flight

#### What is the structure of the dataset?

In [10]:
flights.shape

(31254220, 29)

#### Inspect the data types and missing values

In [11]:
flights.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31254220 entries, 0 to 31254219
Data columns (total 29 columns):
 #   Column             Non-Null Count     Dtype  
---  ------             --------------     -----  
 0   Year               31254220 non-null  int16  
 1   Month              31254220 non-null  int8   
 2   DayofMonth         31254220 non-null  int8   
 3   DayOfWeek          31254220 non-null  int8   
 4   DepTime            30645609 non-null  float16
 5   CRSDepTime         31254220 non-null  int16  
 6   ArrTime            30578778 non-null  float16
 7   CRSArrTime         31254220 non-null  int16  
 8   UniqueCarrier      31254220 non-null  object 
 9   FlightNum          31254220 non-null  int16  
 10  TailNum            31211619 non-null  object 
 11  ActualElapsedTime  30578778 non-null  float16
 12  CRSElapsedTime     31252815 non-null  float16
 13  AirTime            30578778 non-null  float16
 14  ArrDelay           30578778 non-null  float16
 15  DepDelay     

**Observation from info():**

- DepTime, CRSDepTime, ArrTime and CRSArrTime have incorrect integer or float data times, it should be datetime.time with format hhmm
- Reason for delay features (Carrier_Delay, WeatherDelay, NASDelay, SecurityDelay and LateAirCraftDelay) can be converted from float to integer to save memory space

In [12]:
# for further programmatic inspection take a sample to avoid performance issues, as we are working with a large dataset > 2GB
sample = np.random.choice(flights.shape[0], 500000, replace=False)
flight_sample = flights.loc[sample,:].copy()
flight_sample.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
509767,2004,1,21,3,826,805,1114,1052,AS,577,N947AS,168,167,153,22,21,PSP,SEA,987,6,9,0,NaN,0,21,0,1,0,0
13260421,2005,11,27,7,1655,1600,1825,1735,WN,555,N330SW,90,95,79,50,55,ABQ,SLC,493,3,8,0,NaN,0,0,0,0,0,50
12850891,2005,10,14,5,2046,1844,2156,1959,EV,4111,N856AS,70,75,53,117,122,GNV,ATL,300,8,9,0,NaN,0,0,0,117,0,0
5446937,2004,10,20,3,1220,1220,1352,1345,WN,603,N762,92,85,50,7,0,BUR,PHX,369,32,10,0,NaN,0,0,0,0,0,0
7307814,2005,1,8,6,800,810,958,1005,OH,5530,N941CA,118,115,98,-7,-10,ABE,CVG,503,4,16,0,NaN,0,0,0,0,0,0


### Missing data
How much missing data do we have in columns and rows ?

#### Missing data in columns

In [ ]:
# Display columns with missing values
na_counts = flight_sample.isna().sum().sort_values(ascending=False) 

# Drop columns with no missing values
na_counts = na_counts[na_counts > 0]

sns.barplot(x=na_counts.index.values, y=na_counts/flight_sample.shape[0]*100, color=BASE_COLOR)
plt.xticks(rotation=90)
yticks = np.arange(0, 101, 20)
ylabels = ['{}%'.format(v) for v in yticks]
plt.ylabel('Missing values')
plt.title('% Missing values in columns')
plt.yticks(yticks, ylabels)

# get current tick locations and labels
locs, labels = plt.xticks()

# for each bar, print a % text at the top of each bar
for loc, label in zip(locs,labels):
    count = na_counts[label.get_text()]/flight_sample.shape[0]*100
    pct_string = '{:0.1f}%'.format(count)
    plt.text(loc, count+1, pct_string, ha='center', color='black', fontsize=6)

plt.show()
# plt.ticklabel_format(style='plain', axis='y')

In [ ]:
# Take a closer look at records where features like 'AirTime' is missing. It looks like when AirTime is missing, ArrDelay, ArrTime, DepTime and DepDelay could also be missing
flight_sample.loc[flight_sample['AirTime'].isna()][:10]

**Observations missing values in Columns:**

- When AirTime is missing, the flight seems to be always cancelled. Is this the case for 'Diverted' flights also ?
- When a flight is NOT cancelled, CancellationCode is NaN. It's not really a missing value, it's just that the flight is not cancelled.

In [ ]:
# Is airtime, and related columns also missing when a flight is diverted?
flight_sample.loc[flight_sample['Diverted'] == 1][:10]

In [ ]:
# Is airtime only missing when a flight is cancelled or diverted? Make there are not other missing errors
# cols = ['AirTime', 'ArrTime', 'ActualElapsedTime', 'DepTime', 'ArrDelay', 'DepDelay']
flight_sample.loc[(flight_sample['AirTime'].isna()) & (flight_sample['Cancelled'] == 0) & (flight_sample['Diverted'] == 0)][:10]

In [ ]:
# Is airtime only missing when a flight is cancelled or diverted? Make there are not other missing errors
# cols = ['AirTime', 'ArrTime', 'ActualElapsedTime', 'DepTime', 'ArrDelay', 'DepDelay']
flight_sample.loc[(flight_sample['DepDelay'].isna()) & (flight_sample['Cancelled'] == 0) & (flight_sample['Diverted'] == 0)][:10]

**Observations missing values in Columns:**

    - When 'AirTime', 'ArrTime', 'ActualElapsedTime', 'DepTime', 'ArrDelay', 'DepDelay' is missing, the flight is always cancelled or diverted. Missing values in these columns are thus ok. 

#### Missing data in rows

In [ ]:
# missing data in rows 
missing_data_in_rows = (flight_sample.isna().mean(axis=1) * 100).sort_values()
missing_data_in_rows.describe([0.25, 0.5, 0.75, 0.85, 0.9, 0.95, 0.99])

In [ ]:
# Investigate deeper the rows with missing values > 20%
missing_data_in_rows = missing_data_in_rows.loc[(missing_data_in_rows > 20)]
flight_sample.loc[missing_data_in_rows.index][:10]

In [ ]:
# Investigate deeper the rows with missing values > 40%
missing_data_in_rows = missing_data_in_rows.loc[(missing_data_in_rows > 40)]
flight_sample.loc[missing_data_in_rows.index][:10]

> **Observation missing values in rows:**

    - Sub categories of reasons for delays have missing values, even if flights are not cancelled or diverted - to be further investigated
    - Rows with >40% missing records are caused by cancelled or diverted flights

In [ ]:
# Lets have a closer look at NaN values in the sub-category delay fields at the end of the dataset. Are these values missing when ArrDelay is not missing too ?
# flight_sample.loc[(flight_sample['WeatherDelay'].isna()) & (~flight_sample['ArrDelay'].isna())][:10]

### Inspect summary statistics

In [ ]:
flight_sample.describe([0.25, 0.5, 0.75, 0.85, 0.9, 0.95, 0.99], include='all')

**Observation from summary statistics:**

1) `AirTime` have unexpected negative values as minimum values, and looks like errors - this needs a deeper look
2) `ArrDelay` and `DepDelay` have unexpected negative values as minimum values, is this an error or does this represent early departures and arrivals ?
3) DepTime, ArrTime have max time beyond 24:00 (e.g. 24:44). CRSDepTime and CRSArrTime seems to be more reliable time fields with a maximum of 24:00 and contain no missing values.
4) There seems to be huge outliers in field Taxiln, AirTime, ArrDelay, DepDelay and CarrierDelay in percentile 99

Lets deeper explore these issues

#### ActualElapsedTime and AirTime have unexpected negative values and looks like errors - this needs a deeper look

In [ ]:
# Have a closer look at records with negative Airtime
airtime_errors = flight_sample.loc[(flight_sample['AirTime'] < 0)]
print('{} out of {} records have AirTime < 0'.format(airtime_errors.shape[0], flight_sample.shape[0]))
airtime_errors.sample(20)

>Seems like these errors are caused by extremely large TaxiIn values. It is unlikely that the aircraft waited 24 days at the airport and are clear outliers

#### ArrDelay and DepDelay have unexpected negative values, is this an error or does this represent early departures and arrivals ?

In [ ]:
# Have a closer look at records with negative ArrDelay values
airdelay_errors = flight_sample.loc[flight_sample['ArrDelay'] <= 0]
print('{} out of {} records have ArrDelay < 0'.format(airdelay_errors.shape[0], flight_sample.shape[0]))
airdelay_errors.sample(5)

> Due to the volume of records with negative values (+/- 50%), I assume this represent early arrivals and departures

#### DepTime, ArrTime have max time beyond 24:00 (e.g. 24:44)

In [ ]:
# How many records have incorrect time values ?
time_errors = flight_sample.loc[(flight_sample['DepTime'] > 2400) | (flight_sample['ArrTime'] > 2400) | (flight_sample['CRSArrTime'] > 2400)]
print('{} out of {} records have DepTime or ArrTime > 24:00'.format(time_errors.shape[0], flight_sample.shape[0]))
time_errors.head(15)

**Observation**: AirTime is negative when DepTime or ArrTime > 24:00, and again the large `Taxiln` values are present is most cases

### Inspect unique entries

In [ ]:
# number of unique records per feature
flight_sample.nunique()

In [ ]:
# display unique values for non-numeric fields
cols = ['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest', 'Cancelled', 'CancellationCode', 'Diverted']
for col in cols:
    print('Unique values for column {}: \n {} \n'.format(col, flight_sample[col].sort_values().unique()))

**Observation unique():**
    
No errors are detected when inspecting unique values, except for CancellationCode which is not really a missing value, not non-cancelled flights

### Validate time columns 

- Make sure hours are in range 00 - 23
- Make sure minutes are in range 00 - 59

In [ ]:
# display unique hours and minutes
def validate_time(col):
    hours = []
    minutes = []
    
    col =  col.astype('str').str.replace('.0', '', regex=False).str.zfill(4).replace('0nan', None)    
    for time in col:
        if not time == None:
            hour = time[:2]
            minute = time[2:]
            hours.append(hour)
            minutes.append(minute)

    print('\nUnique hours for {}:'.format(col.name))
    hours = set(hours)
    hours = sorted(hours, key=lambda x: (x is None, x))
    print(hours) 
    
    print('\nUnique minutes for {}:'.format(col.name))
    minutes = set(minutes)
    minutes = sorted(minutes, key=lambda x: (x is None, x))
    print(minutes)     

In [ ]:
validate_time(flight_sample['ArrTime'])

In [ ]:
validate_time(flight_sample['CRSArrTime'])

In [ ]:
validate_time(flight_sample['DepTime'])

In [ ]:
validate_time(flight_sample['CRSDepTime'])

**Observation:**
    
- ArrTime and DepTime have incorrect in hours > 24 and incorrect minutes = 60. This will cause problems when converting to datetime format
- CRSArrTime and CRSDepTime have valid hours and minutes and are ready for convertion to datetime format

### Inspect numeric fields  

#### Does ArrDelay add up to the sub category delays ?

In [ ]:
# Is ArrDelay the total of `Carrier_Delay`, `WeatherDelay`, `NASDelay`, `SecurityDelay` and `LateAirCraftDelay` ?

flight_sample['totalDelay'] = flight_sample.CarrierDelay + flight_sample.WeatherDelay + flight_sample.NASDelay + flight_sample.SecurityDelay + flight_sample.LateAircraftDelay
flight_sample.loc[(flight_sample['totalDelay'] - flight_sample['ArrDelay'] == 0 ) & (flight_sample['ArrDelay'] > 0)].shape

Yes, ArrDelay is the total of `CarrierDelay`, `WeatherDelay`, `NASDelay`, `SecurityDelay` and `ateAircraftDelay`in a lot cases

In [ ]:
# When is ArrDelay NOT the total of `Carrier_Delay`, `WeatherDelay`, `NASDelay`, `SecurityDelay` and `LateAirCraftDelay` ?
flight_sample.loc[(flight_sample['totalDelay'] - flight_sample['ArrDelay'] != 0 ) & (flight_sample['ArrDelay'] > 0)][:5]

When ArrDep contain small numbers, then `Carrier_Delay`, `WeatherDelay`, `NASDelay`, `SecurityDelay` and `LateAirCraftDelay` are 0 or NaN.
According to the documentation, a late flight is defined as a flight arriving or departing 15 minutes or more after the scheduled time. Lets investigate this deeper

In [ ]:
# Where ArrDelay > 15 minutes, are there cases where totalDelay does not add to teh sub-categories of `Carrier_Delay`, `WeatherDelay`, `NASDelay`, `SecurityDelay` and `LateAirCraftDelay` ?
flight_sample.loc[(flight_sample['totalDelay'] - flight_sample['ArrDelay'] != 0 ) & (flight_sample['ArrDelay'] > 15)]

**Observation:**

- Assumption made: `ArrDelay` are sometimes negative, which do not represent a delay but rather an early arrival
- `ArrDelay` is always total of `Carrier_Delay`, `WeatherDelay`, `NASDelay`, `SecurityDelay` and `LateAirCraftDelay`, but only when `ArrDelay` is > 15 minutes

## Summary of assessment

### Quality issues
Do we have all the records that we should? Do we have missing records or not? Are there specific rows, columns, or cells missing?

<blockquote>
    
#### Completeness Issues (e.g. missing data)

1) Sub categories of reasons for delays (`Carrier_Delay`, `WeatherDelay`, `NASDelay`, `SecurityDelay`, `LateAirCraftDelay`) contain NaN values, however are not really missing. No delay took place, or delay < 15 minutes

#### Validity Issues (incorrect entries, data types)

2) Columns Cancelled & Diverted are a boolean fields

3) Column `DepTime`, `ArrTime` have max hours beyond 24:00 (e.g. 24:44)
4) AirTime have unexpected negative values. These errors are caused by extremely large TaxiIn values of 24 days. It is unlikely that the aircraft waited 24 days at the airport and are clear outliers
5) Time columns DepTime, CRSDepTime, ArrTime, CRSArrTime, with format hhmm, was converted to datatype float
6) ArrDelay and DepDelay have negative values, which represent early departures or arrivals and not delays
7) Reason for delay features (Carrier_Delay, WeatherDelay, NASDelay, SecurityDelay and LateAirCraftDelay) can be converted from float to integer to save memory space
    
</blockquote>

### Tidiness issues

<blockquote>
    
8) Column headings start with a capital letter
    
</blockquote>

<span style="color:blue; font-size:25px; line-height:30px; font-family: Calibri;">Step 3: Cleaning</span>

In [ ]:
# Make a copy of the original data
flight_clean = flights.copy() 

In [ ]:
# clear the garbage to free memory
gc.collect()

#### Issue #1: Sub categories of reasons for delays (Carrier_Delay, WeatherDelay, NASDelay, SecurityDelay, LateAirCraftDelay) contain NaN values, however are not really missing. No delay took place, or delay < 15 minutes.

**Define**: Fill missing values with 0 for columns Carrier_Delay, WeatherDelay, NASDelay, SecurityDelay, LateAirCraftDelay

##### Code

In [ ]:
cols_delay_reasons = ['CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']

In [ ]:
flight_clean[cols_delay_reasons] = flight_clean[cols_delay_reasons].fillna(0, axis=1)

##### Test

In [ ]:
# sub categories for reasons of delay should have no more missing values
assert (flight_clean[cols_delay_reasons].isna().sum().sum()) == 0

#### Issue #2: Columns Cancelled & Diverted are a boolean fields

**Define**: Convert columns `Cancelled` and `Diverted` to boolen using method astype()

##### Code

In [ ]:
flight_clean['Cancelled'] = flight_clean['Cancelled'].astype('bool')
flight_clean['Diverted'] = flight_clean['Diverted'].astype('bool')

##### Test

In [ ]:
# fields should be converted to type 'bool'
flight_clean[['Cancelled', 'Diverted']].info()

#### Issue #3: Column `DepTime`, `ArrTime` have max hours beyond 24:00 (e.g. 24:44)

**Define**: Filter columns where DepTime and ArrTime <= 2400. As time is in local time, and time zone is not supplied, it's not possible to try to correct these problems.

##### Code

In [ ]:
# take a snapshot of records before
before = flight_clean.shape[0]

In [ ]:
# get rows to drop where hours are >= 2400
rows_to_drop = flight_clean.loc[(flight_clean['DepTime'] >= 2400) | (flight_clean['ArrTime'] >= 2400) | (flight_clean['CRSArrTime'] >= 2400)]

flight_clean.drop(index=rows_to_drop.index, inplace=True)

In [ ]:
# How many records did we drop ?
print('{} out of {} records were dropped'.format(before - flight_clean.shape[0], flight_clean.shape[0]))

##### Test

In [ ]:
# make sure we have not more records with incorrect time > 24:00
assert len(flight_clean.loc[(flight_clean['DepTime'] > 2400) | (flight_clean['ArrTime'] > 2400)]) == 0

#### Issue #4: AirTime have unexpected negative values

**Define**: Filter records where 'AirTime' > 0. This should get rid of negative values in column 'ActualElapsedTime ' as well

##### Code

In [ ]:
# take a snapshot of records before
before = flight_clean.shape[0]

In [ ]:
# get rows to drop
rows_to_drop = flight_clean.loc[flight_clean['AirTime'] < 0]
rows_to_drop.shape

In [ ]:
flight_clean.drop(index=rows_to_drop.index, inplace=True)

##### Test

In [ ]:
assert len(flight_clean.loc[flight_clean['AirTime'] < 0]) == 0

In [ ]:
# How many records did we drop ?
print('{} out of {} records were dropped'.format(before - flight_clean.shape[0], flight_clean.shape[0]))

#### Issue #5: Time columns DepTime, CRSDepTime, ArrTime, CRSArrTime, with format hhmm, was converted to datatype float

**Define**: First converting each column to string and left pad it with leading 0's using astype() and zfill() methods. Convert colums to datetime format using to_datetime()

##### Code

In [ ]:
# before conversion, save snapshot of first records
index = flight_clean.head(3).index
flight_clean.loc[index]

In [ ]:
# Take a snapshot of missing data in ArrTime to make sure it still exist after conversion
missing_before = flight_clean['ArrTime'].isna().sum()

In [ ]:
# def convert_time2(col):

#     # Convert column to string, remove any float decimals, pad with leading zero's and correct missing fields converstions
#     col =  col.astype('str').str.replace('.0', '', regex=False).str.zfill(4).replace('0nan', None)
    
# #     col = col.apply(lambda x: "{}{}".format(str(x), '00'))    
    
#     # Some minute fields are incorrect, covert minute = 60 to minute = 59
#     col  = np.where(col.str[-2:] == '60',  col .str[:2] + '59', col)  
    
# #     col = ':00'.join(col)
    

#     # convert column to datetime
# #     col = pd.to_datetime(col , format='%H%M') 
# #     col = pd.to_timedelta(col, errors='raise')
#     col = pd.to_datetime(col, format='%H%M') - pd.to_datetime(col, format='%H%M').normalize()
    
#     return col

In [ ]:
# flight_clean['ArrTime'] = convert_time2(flight_clean['ArrTime'])

In [ ]:
# flight_clean['ArrTime'] = convert_time(flight_clean['ArrTime']) 
# flight_clean['CRSDepTime'] = convert_time(flight_clean['CRSDepTime']) 
# flight_clean['CRSArrTime'] = convert_time(flight_clean['CRSArrTime']) 
# flight_clean['DepTime'] = convert_time(flight_clean['DepTime']) 

In [ ]:
# change dtype to datetime
flight_clean['DepTime'] = (pd.to_timedelta(flight_clean['DepTime'] // 100, unit='h') +\
                   pd.to_timedelta(flight_clean['DepTime'] % 100, unit='m'))

flight_clean['CRSDepTime'] = (pd.to_timedelta(flight_clean['CRSDepTime'] // 100, unit='h') +\
                   pd.to_timedelta(flight_clean['CRSDepTime'] % 100, unit='m'))

flight_clean['ArrTime'] = (pd.to_timedelta(flight_clean['ArrTime'] // 100, unit='h') +\
                   pd.to_timedelta(flight_clean['ArrTime'] % 100, unit='m'))

flight_clean['CRSArrTime'] = (pd.to_timedelta(flight_clean['CRSArrTime'] // 100, unit='h') +\
                   pd.to_timedelta(flight_clean['CRSArrTime'] % 100, unit='m'))

##### Test

In [ ]:
flight_clean.loc[index]

In [ ]:
# make sure missing before and missing after in ArrTime is still the same
missing_after = flight_clean['ArrTime'].isna().sum()
assert missing_before == missing_after

In [ ]:
# make sure time colums are now type datetime
flight_clean.info()

#### Issue #6: ArrDelay and DepDelay have negative values, which represent early departures or arrivals and not delays

**Define**: Reset negative delays to 0

##### Code

In [ ]:
gc.collect()

In [ ]:
flight_clean['ArrDelay'] = np.where(flight_clean['ArrDelay'] < 0, 0, flight_clean['ArrDelay'])
flight_clean['DepDelay'] = np.where(flight_clean['DepDelay'] < 0, 0, flight_clean['DepDelay'])

##### Test

In [ ]:
assert len(flight_clean.query('ArrDelay < 0')) == 0

In [ ]:
assert len(flight_clean.query('DepDelay < 0')) == 0

#### Issue #7: Reason for delay features (CarrierDelay, WeatherDelay, NASDelay, SecurityDelay and LateAirCraftDelay) can be converted from float to integer to save memory space

**Define**: Convert columns Carrier_Delay, WeatherDelay, NASDelay, SecurityDelay and LateAircraftDelay to int16 using method astype()

##### Code

In [ ]:
# get all columns of type float
# float_columns = list(flight_clean.select_dtypes(include=['float16']).columns)
# float_columns

In [ ]:
# convert all float columns to Int16, as some of the columns contains missing values thus needs to be nullable
# flight_clean[float_columns] = flight_clean[float_columns].astype('Int16')

##### Test

In [ ]:
# fields should be converted to type 'Int16'
# flight_clean[float_columns].info()

#### Issue #8: Column headings start with a capital letter

**Define**: Change column headings to start with lowercase, unless it start with an abbreviation

##### Code

In [ ]:
# only change columns where 2nd character is lowercase and not an abbreviation in capital letters
cols_to_change = [i for i in flight_clean.columns if i[2].islower()] 

In [ ]:
new_colnames = []

for col in flight_clean.columns:
    if col[2].islower():
        col = col[0].lower()+col[1:]
    else:
        col = col
        
    new_colnames.append(col)

In [ ]:
flight_clean.columns = new_colnames

##### Test

In [ ]:
flight_clean.columns

<span style="color:blue; font-size:25px; line-height:30px; font-family: Calibri;">Step 4: Save cleaned data</span>

In [ ]:
flight_clean.shape

In [ ]:
31254220 - 31218803

In [ ]:
# final look at data types
flight_clean.info()

In [ ]:
# save the consolidated cleaned file as pickle file to preserve the datatype corrections
flight_clean.to_pickle(FILE_NAME_CLEAN)

### What is the structure of your dataset?

> The cleaned dataset have 31218803 rows and 29 columns

35417 records was dropped due to incorrect time formats and large negative AirTime caused by large taxiIn values

### What is/are the main feature(s) of interest in your dataset?

> This analysis will concentrate to discover the reasons for delays or cancellations

For delays, the target column to be predicted would be ArrDelay, as this would summarize the total delay both of departure and arrival

For cancellations and diversions, the target columns would be cancelled and diverted

### What features in the dataset do you think will help support your investigation into your feature(s) of interest?

Correlation between the following features could be investigated to determine which predict the target variables the best
- Departure and arrival destinations
- Carriers
- Seasonal trends (e.g. winter vs summer)
- Days of the week


Questions to answer:
- When is the best time of day/day of week/time of year to fly to minimise delays?
- Does certain destigations/carriers cause the delays

- Can you detect cascading failures as delays in one airport create delays in others? Are there critical links in the system?

- How does the number of people flying between different locations change over time?
- How well does weather predict plane delays?

- Do older planes suffer more delays?

### What is the structure of your dataset?

> The cleaned dataset have 31218803 rows and 29 columns

35417 records was dropped due to incorrect time formats and large negative AirTime caused by large taxiIn values

### What is/are the main feature(s) of interest in your dataset?

> This analysis will concentrate to discover the reasons for delays or cancellations

For delays, the target column to be predicted would be ArrDelay, as this would summarize the total delay both of departure and arrival

For cancellations and diversions, the target columns would be cancelled and diverted

### What features in the dataset do you think will help support your investigation into your feature(s) of interest?

Correlation between the following features could be investigated to determine which predict the target variables the best
- Departure and arrival destinations
- Carriers
- Seasonal trends (e.g. winter vs summer)
- Days of the week


Questions to answer:
- When is the best time of day/day of week/time of year to fly to minimise delays?
- Does certain destigations/carriers cause the delays

- Can you detect cascading failures as delays in one airport create delays in others? Are there critical links in the system?

- How does the number of people flying between different locations change over time?
- How well does weather predict plane delays?

- Do older planes suffer more delays?